Goals:

1. Scrape Yahoo Fantasy Rosters Page
Clean Data

Put Into Excel Automatically
Vlookup players from page, create team sheets for each team
Team sheets should show
- Total Predicted Stats for each stat (Maybe z score?)


# 1. Scrape Yahoo Rosters Page

In [173]:
import requests
import bs4


In [174]:
url = 'https://basketball.fantasysports.yahoo.com/nba/82686/startingrosters'


In [175]:
req = requests.get(url)
html_content = req.text

In [176]:
# Parse HTML content
soup = bs4.BeautifulSoup(html_content, 'html.parser')

In [177]:
# Find each team based on the provided HTML structure
teams = soup.find_all('p', class_='W-100 Fz-med Ta-c')

team_data = {}

# strip out the team name
#for team in teams:


for team in teams:

    # Extract team name
    team_name = team.find('a').text.strip()

    # Extract Players

    # Initialize the list of players for this team
    players = []

    # Assuming players are listed within the same parent element or a known structure
    # Here, we assume players are in a sibling <table> element with class 'players-table'
    players_table = team.find_next('table', class_="Table Table-px-med Mbot-xl")

    if players_table:
        player_rows = players_table.find_all('tr', class_=['odd', 'even'])
        for row in player_rows:
            player_name_tag = row.find('a', class_='Nowrap name F-link playernote')
            if player_name_tag:
                player_name = player_name_tag.text.strip()
                players.append(player_name)

    # Add the team and its players to the dictionary
    team_data[team_name] = players

# Print the team data
for team, players in team_data.items():
    print(f'Team: {team}')
    for player in players:
        print(f'{player}')
    print()

# Print number of items in each key
for team, players in team_data.items():
    print(f'Team: {team} - {len(players)} players')


Team: Arthur's Moneyball Team
Fred VanVleet
Donovan Mitchell
Jamal Murray
Amen Thompson
Jabari Smith Jr.
Evan Mobley
Ivica Zubac
Dereck Lively II
Mike Conley
Austin Reaves
Trae Young
Keyonte George
Dyson Daniels
Khris Middleton

Team: Filip's Fantastic Team
Terry Rozier
CJ McCollum
Lonzo Ball
Paolo Banchero
Chet Holmgren
Desmond Bane
Victor Wembanyama
Rudy Gobert
Dennis Schröder
Brandon Ingram
Kelly Oubre Jr.
OG Anunoby
Bennedict Mathurin
Kristaps Porziņģis

Team: Axel's Agreeable Team
Jaden Ivey
Jrue Holiday
Stephon Castle
DeMar DeRozan
Jalen Johnson
Keldon Johnson
Domantas Sabonis
Bam Adebayo
Coby White
Chris Paul
Cam Thomas
Shai Gilgeous-Alexander
Mark Williams
Shaedon Sharpe

Team: Rajon Rondo
Damian Lillard
Christian Braun
Anfernee Simons
Jayson Tatum
Alex Sarr
Karl-Anthony Towns
Myles Turner
Jarrett Allen
Jalen Suggs
Clint Capela
De'Anthony Melton
Jordan Clarkson
Jonathan Kuminga
Devin Vassell
Brandon Miller

Team: Shanghai Sharks
Cade Cunningham
Ochai Agbaji
Scottie Barnes
Julia

# 2. Auto Clean from CSV File

Scrape Site

In [178]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from io import StringIO
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

# Initialize Selenium WebDriver
driver = webdriver.Chrome()  # Make sure you have ChromeDriver installed
driver.get("https://hashtagbasketball.com/fantasy-basketball-projections")

# Allow time for the page to load
time.sleep(5)  # Adjust this as necessary

# Select Dropdown Option
# Find the dropdown menu and select the desired option
dropdown = Select(driver.find_element(By.XPATH, "//select[@id='ContentPlaceHolder1_DDSHOW']"))  # Adjust with the actual class or ID
dropdown.select_by_visible_text("All")  # Change "All" to your desired option

# Wait for page to update
time.sleep(5)




# Parse HTML with BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Find the table and extract data
table = soup.find('table', id="ContentPlaceHolder1_GridView1")  # id to find exact table

# Wrap the HTML string in a StringIO object
table_html = str(table)
table_io = StringIO(table_html)

# Convert table to DataFrame
df = pd.read_html(table_io)[0]

# Close the Selenium driver
driver.quit()

# Save to CSV
df.to_csv("Datasets/Scraped from HashtagBasketball/fantasy_basketball_projections.csv", index=False)

print("Data saved to Datasets/fantasy_basketball_projections.csv")

Data saved to Datasets/fantasy_basketball_projections.csv


Clean df

Remove subheaders
split FG% and FT%
Remove 

In [179]:
# Data cleaning
df = pd.read_csv('Datasets/Scraped from HashtagBasketball/fantasy_basketball_projections.csv')

# Remove Subheaders
df = df[df['POS'] != "POS"]

# Split columns 
df[["FG%", "FGA"]] = df['FG%'].str.split(' ', expand=True)
df[["FT%", "FTA"]] = df['FT%'].str.split(' ', expand = True)

df['FGA'] = df['FGA'].str.extract(r'/([\d.]+)') # [\d.]+  is "one or more digits / periods"
df['FTA'] = df['FTA'].str.extract(r'/([\d.]+)')



# Adjust types 
df = df.astype({
    'PLAYER': str,
    'POS': str,
    'TEAM': str,
    'GP': int,
    'MPG': float,
    'FG%': float,
    'FT%': float,
    '3PM': float,
    'PTS': float,
    'TREB': float,
    'AST': float,
    'STL': float,
    'BLK': float,
    'TO': float,
    'TOTAL': float,
    'FGA': float,
    'FTA': float
})


# Check number of rows
print(f"Number of rows: {len(df)}")
print(f"Number of columns: {len(df.columns)}")

df.to_csv("Datasets/Scraped from HashtagBasketball/fantasy_basketball_projections_cleaned.csv", index=False)

df.head()

Number of rows: 447
Number of columns: 19


,R#,ADP,PLAYER,POS,TEAM,GP,MPG,FG%,FT%,3PM,PTS,TREB,AST,STL,BLK,TO,TOTAL,FGA,FTA
0,1,2.0,Victor Wembanyama,C,SA,70,31.1,0.473,0.802,2.0,24.1,11.6,4.4,1.4,3.9,3.4,13.89,18.6,5.6
1,2,2.3,Nikola Jokic,C,DEN,75,34.1,0.585,0.819,1.1,26.7,12.1,9.0,1.4,0.9,3.0,13.87,17.8,5.8
2,3,4.5,Shai Gilgeous-Alexander,PG,OKC,73,33.5,0.524,0.880,1.4,29.9,5.1,6.5,1.9,0.9,2.4,13.34,19.5,9.2
3,4,3.7,Luka Doncic,"PG,SG",DAL,68,36.8,0.491,0.792,4.1,34.4,9.3,9.9,1.4,0.5,3.8,12.99,23.0,9.7
4,5,7.2,Anthony Davis,"PF,C",LAL,67,34.5,0.559,0.802,0.4,25.2,12.7,3.6,1.2,2.3,2.1,11.92,17.0,7.3


# 3. Refer to Hashtag Basketball CSV File, create CSV files for every team

Use top 200 players to calculate Z scores
Use all players when looking up predicted stats

Use Dictionary of Scraped Teams
Join with data set 
Create 12 different tables 

In [240]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Rename team names in team_data
# Dictionary to map old keys to new keys
rename_map = {
    "Arthur's Moneyball Team": "Arthur",
    "Filip's Fantastic Team": "Filip",
    "Axel's Agreeable Team": "Axel",
    'Rajon Rondo': 'Raj',
    'Shanghai Sharks': 'Asher',
    "Michael's big brains": "Michael",
    'I am inevitable': 'Sykes',
    'Jimmy Butler and Tomfoolery': 'Sarthak',
    "Felix's First-Class Team": "Felix",
    "Guy's team": "Guy",
    "I want Lamelo’s Balls": 'Miles',
    "Josh's Primo Team": "Josh"
}

teams_dic = {rename_map.get(k, k): v for k, v in team_data.items()}

# print number of items in key "Asher"
print(f"Number of items in key 'Asher': {len(teams_dic['Asher'])}")

# From dictionary create a DataFrame of rostered players 
players_teams_dic = []
for team, players in teams_dic.items():
    for player in players:
        players_teams_dic.append({'Player': player, 'TEAM': team})
# Convert to dataframe
rostered_players_df = pd.DataFrame(players_teams_dic)


# Fuzzy merge the two dataframes
database = pd.read_csv('Datasets/Scraped from HashtagBasketball/fantasy_basketball_projections_cleaned.csv')

merged_df = rostered_players_df.merge(database, left_on='Player', right_on='PLAYER', how='left')


# Fuzzy merge for missing values

# Step 1: Identify rows with missing values after initial merge
missing_matches = merged_df[merged_df['PLAYER'].isnull()]

# Step 2: Use fuzzy matching to find close matches in 'database'
for index, row in missing_matches.iterrows():
    player_name = row['Player']
    # Find the closest match in the 'PLAYER' column of the database
    closest_match = process.extractOne(player_name, database['PLAYER'])
    # Step 3: Check if the match meets the threshold (e.g., 90)
    if closest_match[1] >= 80:
        # Update 'Player' with the closest match name for re-merging
        rostered_players_df.loc[rostered_players_df['Player'] == player_name, 'Player'] = closest_match[0]
    else:
        print(f"No close match found for '{player_name}'")

# Step 4: Re-merge after updating names
merged_df = rostered_players_df.merge(database, left_on='Player', right_on='PLAYER', how='left')

merged_df.to_csv("Datasets/Scraped from HashtagBasketball/fantasy_basketball_projections_all_teams.csv", index=False)

# Check for NaN values
print(merged_df.isnull().sum())

# check number of rows for each value of 'TEAM_x'
merged_df['TEAM_x'].value_counts()


Number of items in key 'Asher': 14


TEAM_x
Raj        15
Sykes      15
Filip      14
Arthur     14
Axel       14
Asher      14
Michael    14
Josh       14
Sarthak    13
Felix      13
Guy        13
Miles      13
Name: count, dtype: int64

In [337]:
df = pd.read_csv("Datasets/Scraped from HashtagBasketball/fantasy_basketball_projections_all_teams.csv")

# Drop unnecessary columns
df = df.drop(columns=['PLAYER'])


# Drop Extra rows (Limit to 12) Drop columns with highest R#

df = df.sort_values('R#', ascending = True).groupby('TEAM_x').head(12) # sort by R# and drop the last 12
df['TEAM_x'].value_counts() # check number of rows for each value of 'TEAM_x'


# Make any changes, z score etc.

# z scores to calculate for
stats = ['FG%', 'FT%', '3PM', 'PTS', 'TREB', 'AST', 'STL', 'BLK', 'TO']

database = pd.read_csv('Datasets/Scraped from HashtagBasketball/fantasy_basketball_projections_cleaned.csv')
database['R#'] = database['R#'].astype(str).str.extract(r'(\d+)').astype(int)
database = database.sort_values('R#', ascending=True)[database['R#'] <= 200]

# Calculate FGA Impact and FTA Impact
df['FGA Impact'] = ((df['FG%'] - database['FG%'].mean()) * database['FGA']).round(2)
df['FTA Impact'] = ((df['FT%'] - database['FT%'].mean()) * database['FTA']).round(2)


# Calculate z-scores (3 decimal places)
for col in stats:
    col_zscore = col + '_zscore'
    df[col_zscore] = ((df[col] - database[col].mean()) / database[col].std()).round(2)
    

# Seperate into sheets for each team, with team totals and averages
# z score stats seperated by team (new df), find stats mean and stdev for eahc team and add to stats_summary


df.head()

,Player,TEAM_x,R#,ADP,POS,TEAM_y,GP,MPG,FG%,FT%,3PM,PTS,TREB,AST,STL,BLK,TO,TOTAL,FGA,FTA,FGA Impact,FTA Impact,FG%_zscore,FT%_zscore,3PM_zscore,PTS_zscore,TREB_zscore,AST_zscore,STL_zscore,BLK_zscore,TO_zscore
20,Victor Wembanyama,Filip,1,2.0,C,SA,70,31.1,0.473,0.802,2.0,24.1,11.6,4.4,1.4,3.9,3.4,13.89,18.6,5.6,-0.30,0.08,-0.32,0.19,0.39,1.44,2.38,0.36,1.51,6.04,2.22
45,Jayson Tatum,Raj,10,9.2,"SF,PF",BOS,75,35.8,0.469,0.836,2.9,27.9,9.0,5.1,1.1,0.7,2.5,8.58,20.1,7.3,-0.28,0.20,-0.38,0.59,1.32,2.11,1.33,0.70,0.49,0.01,0.96
13,Khris Middleton,Arthur,100,112.6,SF,MIL,62,28.1,0.482,0.867,1.7,16.4,4.4,4.7,0.8,0.2,2.4,1.35,12.9,2.6,-0.21,0.29,-0.18,0.96,0.08,0.08,-0.53,0.51,-0.53,-0.93,0.82
8,Mike Conley,Arthur,101,134.0,PG,MIN,72,29.1,0.442,0.885,2.2,11.6,2.8,6.0,1.1,0.2,1.4,1.30,8.7,1.9,-1.00,0.88,-0.80,1.18,0.60,-0.76,-1.17,1.13,0.49,-0.93,-0.58
119,Jonas Valanciunas,Felix,102,89.4,C,WAS,74,26.8,0.548,0.796,0.5,14.2,10.3,2.1,0.4,0.8,1.8,1.30,10.4,2.9,0.45,0.01,0.85,0.11,-1.17,-0.31,1.86,-0.74,-1.89,0.20,-0.02


In [404]:
zstats = [col + '_zscore' for col in stats]

# Calculate means and standard deviations for each stat
zstats_summary = pd.DataFrame({
    'zStat': zstats
    #'Global Mean': [database[col].mean().round(2) for col in stats],
    #'Global StdDev': [database[col].std().round(2) for col in stats]
})

zstats_summary.index = zstats_summary['zStat']
zstats_summary = zstats_summary.drop(columns='zStat')



# Create a new DataFrame for each team with z-scores
database_with_zscores = df.copy().drop(columns=['TOTAL', 'ADP'])


with pd.ExcelWriter('Datasets/Yahoo Team Data/Yahoo Teams.xlsx') as writer:
    for team in database_with_zscores['TEAM_x'].unique():
        team_df = database_with_zscores[database_with_zscores['TEAM_x'] == team] # filter by team
        
        #save to excel
        team_df.to_excel(writer, sheet_name=team, index=False)

        # Calculate sum of z-scores for each stat
        zscore_sum = team_df[zstats].sum(axis=0).round(2)
        zstats_summary[team] = zscore_sum
    # Remove "_zscore" from column names
    zstats_summary.index = zstats_summary.index.str.replace('_zscore', '')
    zstats_summary.T.to_excel(writer, sheet_name='Summary', index=True)
print("Data saved to Datasets/Yahoo Team Data/Yahoo Teams.xlsx")




    #print(f"Team: {team}")
    #print(team_means)


    #team_df['TOTAL_zscore'] = team_df[[col + '_zscore' for col in stats]].sum(axis=1).round(2)
    #team_df['TOTAL'] = team_df[stats].sum(axis=1)
    #team_df['ADP'] = team_df['R#'].rank(ascending=True).astype(int)
    #team_df.to_csv(f"Datasets/Yahoo Team Data/{team}_stats.csv", index=False)


# Create a new DataFrame for each team with z-scores
# stats = ['FG%', 'FT%', '3PM', 'PTS', 'TREB', 'AST', 'STL', 'BLK', 'TO']
# Df column names are "STAT" and then team names 

print(zscore_sum.index)
print(zstats_summary.index)


zstats_summary.head(15)

Data saved to Datasets/Yahoo Team Data/Yahoo Teams.xlsx
Index(['FG%_zscore', 'FT%_zscore', '3PM_zscore', 'PTS_zscore', 'TREB_zscore',
       'AST_zscore', 'STL_zscore', 'BLK_zscore', 'TO_zscore'],
      dtype='object')
Index(['FG%', 'FT%', '3PM', 'PTS', 'TREB', 'AST', 'STL', 'BLK', 'TO'], dtype='object', name='zStat')


,Filip,Raj,Arthur,Felix,Asher,Guy,Miles,Sykes,Michael,Axel,Josh,Sarthak
zStat,,,,,,,,,,,,
FG%,-2.75,-0.15,-1.37,-2.76,-2.45,7.33,1.56,-4.70,-4.38,1.08,-0.78,-0.18
FT%,1.23,0.43,2.66,4.53,-1.17,0.83,3.25,0.31,3.97,0.31,1.87,-4.06
3PM,0.72,-0.41,4.48,6.13,-0.10,-3.63,-0.01,4.68,2.62,-5.38,2.83,-1.53
PTS,3.65,2.15,1.69,3.31,0.58,2.08,5.45,5.44,3.25,2.20,-0.06,1.82
TREB,2.65,4.35,-3.86,2.38,-0.21,3.10,-1.16,-2.09,-3.53,3.22,-0.90,1.92
AST,-0.78,-3.93,8.19,6.34,-0.72,-3.53,2.39,5.17,-0.78,3.25,-0.10,6.03
STL,-0.92,-4.66,3.84,-0.24,-0.24,-6.36,-2.96,5.20,-1.94,3.84,-1.60,0.78
BLK,8.77,4.62,-1.96,-4.42,3.68,5.37,-4.05,-6.32,-5.01,-1.40,-1.99,-3.10
TO,3.54,0.32,4.80,3.26,-0.38,-0.52,3.83,6.06,2.42,2.28,-4.72,2.14


In [401]:

zstats_summary.T.head(15)

zStat,FG%_zscore,FT%_zscore,3PM_zscore,PTS_zscore,TREB_zscore,AST_zscore,STL_zscore,BLK_zscore,TO_zscore
Filip,-2.75,1.23,0.72,3.65,2.65,-0.78,-0.92,8.77,3.54
Raj,-0.15,0.43,-0.41,2.15,4.35,-3.93,-4.66,4.62,0.32
Arthur,-1.37,2.66,4.48,1.69,-3.86,8.19,3.84,-1.96,4.80
Felix,-2.76,4.53,6.13,3.31,2.38,6.34,-0.24,-4.42,3.26
Asher,-2.45,-1.17,-0.10,0.58,-0.21,-0.72,-0.24,3.68,-0.38
Guy,7.33,0.83,-3.63,2.08,3.10,-3.53,-6.36,5.37,-0.52
Miles,1.56,3.25,-0.01,5.45,-1.16,2.39,-2.96,-4.05,3.83
Sykes,-4.70,0.31,4.68,5.44,-2.09,5.17,5.20,-6.32,6.06
Michael,-4.38,3.97,2.62,3.25,-3.53,-0.78,-1.94,-5.01,2.42
Axel,1.08,0.31,-5.38,2.20,3.22,3.25,3.84,-1.40,2.28


In [215]:

# Save each team DataFrame to a separate sheet in an Excel file
with pd.ExcelWriter('team_projections.xlsx') as writer:
    for team, team_df in team_dfs.items():
        team_df.to_excel(writer, sheet_name=team, index=False)
    all_teams_df.to_excel(writer, sheet_name='All Teams', index=False)
print("Data saved to team_projections.xlsx")


Number of items in key 'Asher': 14
Closest match for Kristaps Porziņģis: Kristaps Porzingis
Closest match for Alex Sarr: Alexandre Sarr
Closest match for Jusuf Nurkić: Jusuf Nurkic
Closest match for Nic Claxton: Nicolas Claxton
Closest match for Bogdan Bogdanović: Bogdan Bogdanovic
Closest match for Nikola Jokić: Nikola Jokic
Closest match for Nikola Vučević: Nikola Vucevic
Closest match for P.J. Washington Jr.: P.J. Washington
Closest match for Jonas Valančiūnas: Jonas Valanciunas
Closest match for Luka Dončić: Luka Doncic
Closest match for Alperen Şengün: Alperen Sengün
Data saved to team_projections.xlsx
Number of rows for Asher: 11


# 4 Clean up "ALL TEAMS" csv file


12 players max per team
- Remove lowest rank ("R#")

Convert to z scores
- mean and stdev of top 300 players sample

Find team totals of all z-scores 

In [197]:
df = pd.read_csv('Datasets/Scraped from HashtagBasketball/fantasy_basketball_projections_all_teams.csv')

# group by TEAM, sort by "R#" and only keep 12 lowest "R#"
df = df.sort_values(by=["TEAM", "R#"], ascending=[True, False])

# Keep only the top 12 rows for each "TEAM"
df = df.groupby("TEAM").head(12)

# group by TEAM and show number of rows in each group
df.groupby("TEAM").size()



,R#,ADP,PLAYER,POS,TEAM,GP,MPG,FG%,FT%,3PM,PTS,TREB,AST,STL,BLK,TO,TOTAL,FGA,FTA
55,17,18.5,Scottie Barnes,"SG,SF,PF",Asher,70,34.8,0.479,0.791,1.7,19.1,8.3,6.0,1.3,1.5,2.4,7.18,14.5,4.4
56,26,10.0,Anthony Edwards,"SG,SF",Asher,79,35.4,0.460,0.823,2.5,26.2,5.5,5.2,1.4,0.7,3.1,5.96,20.3,6.1
57,30,37.1,Jaren Jackson Jr.,"PF,C",Asher,68,31.2,0.470,0.800,1.7,21.3,6.3,1.7,1.2,1.9,2.1,4.97,15.9,5.8
58,39,53.9,Kawhi Leonard,"SG,SF,PF",Asher,48,30.0,0.507,0.859,1.7,20.2,5.4,3.2,1.1,0.5,1.6,4.27,14.7,4.1
59,45,33.5,Cade Cunningham,"PG,SG",Asher,64,33.7,0.450,0.865,1.9,23.2,4.7,7.8,0.9,0.4,3.5,3.87,19.5,4.4
60,70,86.8,Walker Kessler,C,Asher,69,26.1,0.662,0.577,0.0,9.5,8.7,1.0,0.4,2.5,1.0,2.07,6.3,1.9
61,104,117.0,Tyus Jones,PG,Asher,73,29.4,0.476,0.802,1.7,12.2,2.9,6.8,1.2,0.2,1.1,1.24,10.1,1.1
62,129,108.4,Klay Thompson,"SG,SF",Asher,70,29.4,0.434,0.901,3.4,17.2,3.2,2.0,0.6,0.4,1.4,-0.09,14.3,1.6
63,142,137.7,Rui Hachimura,"SF,PF",Asher,66,29.2,0.514,0.740,1.5,14.8,5.2,1.3,0.7,0.5,0.9,-0.55,11.3,2.3
64,210,NaN,Julian Champagnie,"SF,PF",Asher,60,21.4,0.420,0.810,1.8,9.0,3.6,1.5,0.7,0.6,0.9,-2.43,7.2,1.5


In [198]:
df = pd.read_csv('Datasets/Scraped from HashtagBasketball/fantasy_basketball_projections_all_teams.csv')

df.groupby("TEAM").size()


TEAM
Arthur     14
Asher      11
Axel       14
Felix      12
Filip      13
Guy        13
Josh       12
Michael    14
Miles      13
Raj        14
Sarthak    11
Sykes      14
dtype: int64

In [142]:
# Print the team data
print(team_data.keys())

for team, players in team_data.items():
    print(f'Team: {team}')
    for player in players:
        print(f'{player}')
    print()

dict_keys(["Arthur's Moneyball Team", "Filip's Fantastic Team", "Axel's Agreeable Team", 'Rajon Rondo', 'Shanghai Sharks', "Michael's big brains", 'I am inevitable', 'Jimmy Butler and Tomfoolery', "Felix's First-Class Team", "Guy's team", 'I want Lamelo’s Balls', "Josh's Primo Team"])
Team: Arthur's Moneyball Team
Fred VanVleet
Donovan Mitchell
Jamal Murray
Amen Thompson
Jabari Smith Jr.
Evan Mobley
Ivica Zubac
Dereck Lively II
Mike Conley
Austin Reaves
Trae Young
Keyonte George
Dyson Daniels
Khris Middleton

Team: Filip's Fantastic Team
CJ McCollum
Reed Sheppard
Kelly Oubre Jr.
Paolo Banchero
OG Anunoby
Desmond Bane
Victor Wembanyama
Rudy Gobert
Dennis Schröder
Brandon Ingram
Terry Rozier
Chet Holmgren
Bennedict Mathurin
Kristaps Porziņģis
Lonzo Ball

Team: Axel's Agreeable Team
Jaden Ivey
Jrue Holiday
Stephon Castle
DeMar DeRozan
Jalen Johnson
Keldon Johnson
Domantas Sabonis
Bam Adebayo
Coby White
Chris Paul
Cam Thomas
Shai Gilgeous-Alexander
Mark Williams
Shaedon Sharpe

Team: Rajon

In [135]:
print(team_data.keys())

dict_keys(["Arthur's Moneyball Team", "Filip's Fantastic Team", "Axel's Agreeable Team", 'Rajon Rondo', 'Shanghai Sharks', "Michael's big brains", 'I am inevitable', 'Jimmy Butler and Tomfoolery', "Felix's First-Class Team", "Guy's team", 'I want Lamelo’s Balls', "Josh's Primo Team"])
